In [1]:
import numpy as np
import networkx as nx
import random 
import math
import pandas as pd
import os 
from tqdm import tqdm

In [2]:
from multiprocessing import Pool
import time
import itertools

import matplotlib.pyplot as plt
import networkx as nx
import numba as nb


def chunks(l, n):
    """Divide a list of nodes `l` in `n` chunks"""
    l_c = iter(l)
    while 1:
        x = tuple(itertools.islice(l_c, n))
        if not x:
            return
        yield x


def betweenness_centrality_parallel(G, processes=None):
    """Parallel betweenness centrality  function"""
    p = Pool(processes=processes)
    node_divisor = len(p._pool) * 4
    node_chunks = list(chunks(G.nodes(), int(G.order() / node_divisor)))
    num_chunks = len(node_chunks)
    bt_sc = p.starmap(
        nx.betweenness_centrality_subset,
        zip(
            [G] * num_chunks,
            node_chunks,
            [list(G)] * num_chunks,
            [True] * num_chunks,
            [None] * num_chunks,
        ),
    )

    # Reduce the partial solutions
    bt_c = bt_sc[0]
    for bt in bt_sc[1:]:
        for n in bt:
            bt_c[n] += bt[n]
    return bt_c


In [3]:
graph_dir = 'C:\\Users\\bbb50\\Desktop\\Python\\hw1\\test_graph_20000\\'
bc_val_dir = 'C:\\Users\\bbb50\\Desktop\\Python\\hw1\\test_bc_val_20000\\'

In [ ]:
for i in tqdm(range(24,30)):
    g = nx.powerlaw_cluster_graph(n=20000 , m=4, p=0.05, seed=None)
    bc = betweenness_centrality_parallel(g, processes=8)
    bc_val = [bc[j] for j in bc]
    nx.write_adjlist(g, (graph_dir + str(i)))
    np.save((bc_val_dir + str(i) + '.npy'), np.array(bc_val))

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]